In [53]:
import numpy as np                   # import numpy package under shorthand "np"
import pandas as pd                  # import pandas package under shorthand "pd"
import matplotlib.pyplot as plt
from nose.tools import assert_equal
from sklearn import preprocessing
from numpy.testing import assert_array_equal
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [46]:
df = pd.read_csv('noon_perfumes_dataset.csv')
df.head(5)
#droppin gunsued columns
data = df.drop(['brand', 'name','old_price','new_price','ml','concentration','item_rating','seller','seller_rating','num_seller_ratings'],axis=1)
data = data.drop(data[data['department'] =='Kids Unisex'].index)
date_c=[]
for i in data['base_note']:
    k=i.replace('and ', ', ')
    k=k.replace('And ', ', ')
    k=k.replace(' Notes ', '')
    k=k.lower()
    k=i.replace('and ', ', ')
    k=k.replace('And ', ', ')
    k=k.replace(' Notes ', '')
    k=k.lower()
    k=k.replace(',',' ')
    k=k.replace(', ',' ')
    k=k.replace(' ,',' ')
    k=k.replace('-',' ')
    k=k.replace('.','')
    k=k.replace(' notes',' ')
    k=k.replace(' note',' ')
    k=k.replace(' accords','')
    k=k.replace(' accord','')
    k=k.replace('dsy','dy')
    k=k.replace('a ','')
    k=k.replace('of ','')
    k=k.replace('(oud)','oud')
    k=k.replace(' and ',' ')
    k=k.replace(' with ',' ')
    date_c.append(k)
    
date_m=[]
for i in data['middle_note']:
    k=i.replace('and ', ', ')
    k=k.replace('And ', ', ')
    k=k.replace(' Notes ', '')
    k=k.lower()
    k=k.replace(',',' ')
    k=k.replace(', ',' ')
    k=k.replace(' ,',' ')
    k=k.replace('-',' ')
    k=k.replace('.','')
    k=k.replace(' notes',' ')
    k=k.replace(' note',' ')
    k=k.replace(' accords','')
    k=k.replace(' accord','')
    k=k.replace('dsy','dy')
    k=k.replace('a ','')
    k=k.replace('of ','')
    k=k.replace('(oud)','oud')
    k=k.replace(' and ',' ')
    k=k.replace(' with ',' ')
    date_m.append(k)
    
date_s=[]
for i in data['scents']:
    k=i.replace('Oriental, Floral', 'Floral and Oriental')
    k=k.replace('Rose, Floral', 'Floral')
    k=k.replace(' and ',' ')
    k=k.replace(',',' ')
    k=k.replace(', ',' ')
    date_s.append(k)
data.insert(0,"base",date_c)
data.insert(0,"scent",date_s)
data.insert(0,"middle",date_m)


print(data['scent'].unique())

['Woody' 'Floral' 'Arabian' 'Spicy' 'Oriental' 'Fruity' 'Fresh' 'Citrus'
 'Vanilla' 'Musk' 'Floral Oriental' 'Sweet Aromatic' 'Aromatic'
 'Woody  Sweet' 'Aromatic Citrus' 'Clean' 'Woody  Musky' 'Jasmine'
 'Woody And Spicy' 'Sandalwood']


In [36]:
display(data)

,middle,scent,base,Unnamed: 0,department,scents,base_note,middle_note
0,hazelnut jasmine cashmir wood cedar honey,Woody,oakmoss patchouli vetiver,0,Men,Woody,"Oakmoss, Patchouli and Vetiver","Hazelnut, Jasmine, Cashmir Wood, Cedar and Honey"
1,wild jasmine red lily,Floral,vanill sandalwood patchouli,1,Women,Floral,"Vanilla, Sandalwood And Patchouli",Wild Jasmine and Red Lily
2,sandalwood cedar,Arabian,lemon mint wood moss,2,Unisex,Arabian,"Lemon, Mint and Wood Moss",Sandalwood and Cedar
3,blue coral aquaspace geranium,Spicy,cashmere wood moss rippled s,3,Men,Spicy,"Cashmere Wood, Moss And Rippled Sand Accord",Blue Coral Aquaspace Accord And Geranium
4,african orange flower,Arabian,vanille benzoin tonkbean,4,Women,Arabian,"Vanille, Benzoin, Tonka Bean",African Orange Flower
...,...,...,...,...,...,...,...,...
998,wild rose elegant jasmine,Citrus,coconut wood musk,998,Women,Citrus,"Coconut Accords, Wood, Musk","Wild Rose, Elegant Jasmine"
999,woody floral sandalwood cedar,Fruity,musk vanilla,999,Women,Fruity,Musk And Vanilla,"Woody-Floral, Sandalwood And Cedar"
1000,ylang ylang,Woody,musk amber wood,1000,Women,Woody,"Musk, Amber, Wood",Ylang-ylang
1001,ylang ylang rose jasmine,Arabian,agarwood amber,1001,Unisex,Arabian,Agarwood and Amber,"Ylang-Ylang, Rose and Jasmine"


In [62]:
data['department'].value_counts().head(10)

department
Women     503
Men       433
Unisex     66
Name: count, dtype: int64

In [47]:
data2=data
data2 = data2.drop(['scents','base_note','middle_note','Unnamed: 0'],axis=1)
data2.head()

,middle,scent,base,department
0,hazelnut jasmine cashmir wood cedar honey,Woody,oakmoss patchouli vetiver,Men
1,wild jasmine red lily,Floral,vanill sandalwood patchouli,Women
2,sandalwood cedar,Arabian,lemon mint wood moss,Unisex
3,blue coral aquaspace geranium,Spicy,cashmere wood moss rippled s,Men
4,african orange flower,Arabian,vanille benzoin tonkbean,Women


In [48]:
#One hot encoding

bas=data2['base'].str.get_dummies(sep=' ')

mid=data2['middle'].str.get_dummies(sep=' ')
sce=data2['middle'].str.get_dummies(sep=' ')
frames = [bas,mid,sce]
 
onehot = pd.concat(frames,axis=1, join='inner')
display(onehot)


,aandal,absolute,acaciwood,addiction,agarwood,akigalawood,aldehyde,almod,almond,almonds,...,woods,woody,woodyoakmoss,wormwood,yellow,ylan,ylang,zest,zests,–
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
999,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1001,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [56]:
#getting labels
label_encoder = preprocessing.LabelEncoder()
gender2= label_encoder.fit_transform(data['department'])

#SVC Method
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

cv_results = cross_validate(clf,X_train, y_train, cv=5,return_train_score=True)
print(cv_results)

{'fit_time': array([0.11577392, 0.11458182, 0.11237788, 0.11625767, 0.11589527]), 'score_time': array([0.05102682, 0.05070472, 0.05041218, 0.05119181, 0.05101681]), 'test_score': array([0.80141844, 0.82857143, 0.77857143, 0.82142857, 0.81428571]), 'train_score': array([0.92142857, 0.91622103, 0.92156863, 0.9144385 , 0.92156863])}


In [57]:
#Logistic regression Method
clf2 = LogisticRegression(random_state=0)

cv_results2 = cross_validate(clf2,X_train, y_train, cv=5,return_train_score=True)
print(cv_results2)

{'fit_time': array([0.07506204, 0.07680082, 0.068537  , 0.07060146, 0.07270002]), 'score_time': array([0.0082972 , 0.00876188, 0.00888777, 0.00864649, 0.00914621]), 'test_score': array([0.79432624, 0.83571429, 0.82142857, 0.8       , 0.85714286]), 'train_score': array([0.96071429, 0.96434938, 0.95008913, 0.96791444, 0.96256684])}


In [61]:
#Calculating the averages of the test scores
print(np.mean([0.80141844, 0.82857143, 0.77857143, 0.82142857, 0.81428571]))
print(np.mean([0.79432624, 0.83571429, 0.82142857, 0.8       , 0.85714286]))

print(1-np.mean([0.79432624, 0.83571429, 0.82142857, 0.8       , 0.85714286]))

0.8088551159999999
0.8217223920000001
0.17827760799999992
